In [1]:
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam


from datetime import datetime
import itertools
import argparse
import re
import os
import pickle

from sklearn.preprocessing import StandardScaler


In [9]:
# Use AAPL(Apple), MSI (Motorola), SBUX ( Starbacks)
def get_data():
    # returns a T x 3 list of stock prices
    # each row is a different stock
    # 0 = AAPL
    # 1 = MSI
    #2 - SBUX

    df = pd.read_csv('aapl_msi_sbux.csv')
    return df.values
get_data()

array([[ 67.8542,  60.3   ,  28.185 ],
       [ 68.5614,  60.9   ,  28.07  ],
       [ 66.8428,  60.83  ,  28.13  ],
       ...,
       [156.49  , 101.06  ,  54.69  ],
       [163.03  , 102.76  ,  55.61  ],
       [159.54  , 102.63  ,  54.46  ]])

In [10]:
df.head()

,AAPL,MSI,SBUX
0,67.8542,60.30,28.185
1,68.5614,60.90,28.070
2,66.8428,60.83,28.130
3,66.7156,60.81,27.915
4,66.6556,61.12,27.775


In [11]:
df.shape

(1259, 3)

Experience Relay Memory

In [12]:
class ReplayBuffer:
    def __init__(self, obs_dim, act_dim, size):
        #for state
        self.obs1_buf = np.zeros([size, obs_dim], dtype=np.float32)
        #for next state
        self.obs2_buf = np.zeros([size, obs_dim], dtype=np.float32)
        #unit8 for representing 0-26 inclusive for actions
        self.acts_buf = np.zeros(size, dtype=np.unit8)
        self.rews_buf = np.zeros(size, dtype=np.float32)
        #0 or 1 so unit8
        self.done_buf = np.zeros(size, dtype=np.unit8)
        self.ptr, self.size, self.max_size = 0, 0, size
        
    def store(self, obs, act, rew, next_obs, done):
        self.obs1_buf[self.ptr] = obs
        self.obs2_buf[self.ptr] = next_obs
        self.acts_buf[self.ptr] = act
        self.rews_buf[self.ptr] = rew
        self.done_buf[self.ptr] = done
        #since it is circuler it up to maxsize and fix the pointer to 0
        self.ptr = (self.ptr+1) % self.max_size
        self.size = min(self.size+1, self.max_size)
        
    def sample_batch(self, batch_size = 32):
        #choose random index 0 to the size of buffer and return the values according to the index
        idxs = np.random.randint(0, self.size, size=batch_size)
        return dict(s=self.obs1_buf[idxs],
                    s2=self.obs2_buf[idxs],
                    a=self.acts_buf[idxs],
                    r=self.rews_buf[idxs],
                    d=self.done_buf[idxs])
    
    
def get_scaler(env):
    #return scikit-learn scaler object to scale the states
    #Note: you could also populate the replay buffar here
    state = []
    for _ in range(env.n_step):
        action = np.random.choice(env.action_space)
        state, reward, done, info = enc.step(action)
        states.append(state)
        if done:
            break
            
    scaler = StandardScaler()
    scaler.fit(states)
    return scaler

def maybe_make_dir(directory):
    if not os.path.exists(directory):
        os.mekdirs(directory)
        
##Create Regression model
def mlp(input_dim, n_action, n_hidden_layers=1, hidden_dims=32):
    """multi layer perceptron"""
    
    #input layer
    i = Input(shape=(input_dim,))
    x = i
    
    #hidden layers
    for _ in range(n_hidden_layers):
        x = Dense(hidden_dims, activetion='relu')(x)
    
    #final layer
    x = Dense(n_action)(x)
    
    model = Model(i, x)
    
    model.compile(loss='mse', optimizer='adam')
    print((model.summary()))
    return model

In [ ]:
class MultiStockEnv:
    '''
    A 3-Stock trading environment.
    State: vactor of size 7 ( n_stock * 2 + 1)
        - # share of stock 1 owned
        - # share of stock 2 owned
        - # share of stock 3 owned
        - price of stock 1 (using daily close price)
        - price of stock 2 
        - price of stock 3
        - cash owned (can be used to purchase more stocks)
    Action: categorical variable with 27 (3^3) possibilities
        - for each stock, you can: 
        - 0 = sell
        - 1 = hold
        - 2 = buy
    '''
    
    def __init__(self, data, initial_investment=20000):
        #data
        self.stock_price_history = data
        self.n_step, self.n_stock = self.stock_price_history.shape
        
        #instance attributes
        self.initial_investment = initial_investment
        self.cur_step = None
        self.stock_owned = None
        self.stock_price = None
        
        self.action_space = np.arrange(3**self.n_stock)
        
        '''
        action permutations
        returns a nested list with elements like:
            [0,0,0]
            [0,0,1]
            [0,0,2]
            [0,1,0]
            [0,1,1] 
            etc
        - 0 = sell
        - 1 = hold
        - 2 = buy
        '''
        self.action_list = list(map(list, iterools.product([0,1,2], repeat=self.n_stock)))
        
        #calculate size of state
        self.state_dim = self.n_stock * 2 + 1
        
        self.reset()
        
    def reset(self):
        self.cur_step = 0
        selff.stock_owned = np.zeros(self.n_stock)
        self.stock_price = self.stock_price_history[self.cur_step]
        self.cahs_in_hand = self.initial_investment
        return self._get_obs()
    
    def step(self, action):
        #check the valur exist in the action
        assert action in self.action_space
        
        #get current value before performing the action
        prev_val = self._get_val()
        
        #Update price, i.e. go to the next day
        self.cur_step += 1
        self.stock_price = self.stocl_price_history[self.cur_step]
        
        #perform the trade
        self._trade(action)
        
        #get the new value after taking the action
        cur_val = self._get_val()
        
        #reward is
        
        